In [1]:
from rlrom.testers import RLTester
import rlrom.utils as utils
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
from pprint import pprint
from gymnasium import spaces

Loading BokehJS ...

In [2]:
cfg = utils.load_cfg('cfg0_hug_specs.yml')
#cfg = utils.load_cfg('cfg_min.yml')
pprint(cfg)


loading field [ specs ] from STL file [ cartpole.stl ]
{'cfg_specs': {'action_names': {'push': 'action'},
               'eval_formulas': {'phi_far_left': None},
               'obs_names': {'theta': 'obs[2]',
                             'theta_dot': 'obs[3]',
                             'x': 'obs[0]',
                             'x_dot': 'obs[1]'},
               'specs': 'signal push, x, x_dot, theta, theta_dot, reward\n'
                        '\n'
                        '\n'
                        'go_left := x_dot[t]>0\n'
                        'far_left := x[t]<-0.1\n'
                        'phi_far_left := ev_[0, 100] far_left'},
 'cfg_test': {'init_seeds': 1,
              'num_ep': 1,
              'num_steps': 100,
              'render_mode': 'human'},
 'env_name': 'CartPole-v1',
 'model_name': 'sb3/ppo-CartPole-v1',
 'model_path': 'huggingface'}


In [3]:
T = RLTester(cfg)
Tres= T.run_cfg_test()

== CURRENT SYSTEM INFO ==
- OS: Linux-6.14.0-29-generic-x86_64-with-glibc2.39 # 29~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Aug 14 16:52:50 UTC 2
- Python: 3.12.3
- Stable-Baselines3: 2.7.0
- PyTorch: 2.8.0+cu128
- GPU Enabled: True
- Numpy: 2.3.2
- Cloudpickle: 3.1.1
- Gymnasium: 1.2.0

== SAVED MODEL SYSTEM INFO ==
- OS: Linux-5.15.0-97-generic-x86_64-with-glibc2.35 # 107-Ubuntu SMP Wed Feb 7 13:26:48 UTC 2024
- Python: 3.10.9
- Stable-Baselines3: 2.3.0a3
- PyTorch: 2.2.0+cpu
- GPU Enabled: False
- Numpy: 1.24.4
- Cloudpickle: 3.0.0
- Gymnasium: 0.29.1
- OpenAI Gym: 0.26.2



/home/alex/workspace_local/rlrom/.venv/lib/python3.12/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
/home/alex/workspace_local/rlrom/.venv/lib/python3.12/site-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/alex/workspace_local/rlrom/.venv/lib/python3.12/site-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=f

.


In [ ]:
Tr = T.test_results
print(Tr)

In [4]:
lay = """
_ep(1)
 x
 sat(far_left)
 reward
"""
fig, _=  T.get_fig(lay)
show(fig)

_ep(1)
x
sat(far_left)
reward


/home/alex/workspace_local/rlrom/.venv/lib/python3.12/site-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/alex/workspace_local/rlrom/.venv/lib/python3.12/site-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [ ]:
T.env.env.get_values_from_str('x')